### puzzle 1 ###

In [1]:
import numpy as np

In [2]:
start_pattern = '.#./..#/###'
test_rules = '''../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#'''.split('\n')

In [3]:
test_rules

['../.# => ##./#../...', '.#./..#/### => #..#/..../..../#..#']

In [4]:
def pattern_to_matrix(pattern):
    pt = pattern.split('/')
    nrows = len(pt)
    ncols = len(pt[0])
    arr = np.zeros((nrows, ncols))
    for i,row in enumerate(pt):
        for j,c in enumerate(row):
            if c == '.':
                arr[i][j] = 0
            elif c == '#':
                arr[i][j] = 1
            else:
                raise ValueError('Unexpected character: {}'.format(c))
    return arr

In [5]:
def process_rules(rules):
    rulebook = []
    for rule in rules:
        lhs, arrow, rhs = rule.partition('=>')
        lhs_arr = pattern_to_matrix(lhs.strip())
        rhs_arr = pattern_to_matrix(rhs.strip())
        rulebook.append((lhs_arr, rhs_arr))
    return rulebook

In [6]:
test_rulebook = process_rules(test_rules)

In [7]:
def is_match(a, b):
    if len(a) != len(b):
        return False
    for n in range(4):
        rotb = np.rot90(b, n)
        if np.array_equal(a, rotb):
            return True
        if np.array_equal(a, np.fliplr(rotb)):
            return True
    return False
assert(is_match(pattern_to_matrix(start_pattern), pattern_to_matrix('###/..#/.#.')))

In [8]:
def find_replacement(rulebook, item):
    for lhs,rhs in rulebook:
        if is_match(item, lhs):
            return rhs
    raise ValueError('No rule matches?!')        

In [9]:
def enhance(grid, rulebook):
    size = len(grid)
    if size % 2 == 0:
        blk_size = 2
        new_blk_size = 3
    elif size % 3 == 0:
        blk_size = 3
        new_blk_size = 4
    else:
        raise ValueError('Nonsensical grid size: {}'.format(size))
    num_blocks = len(grid)//blk_size
    # new grid is num_blocks × num_blocks, where each block is new_blk_size×new_blk_size
    newgrid = np.zeros((new_blk_size*num_blocks,new_blk_size*num_blocks))
    for iblk in range(num_blocks):
        for jblk in range(num_blocks):
            curr_block = grid[iblk*blk_size:(iblk+1)*blk_size,jblk*blk_size:(jblk+1)*blk_size]
            newgrid[iblk*new_blk_size:(iblk+1)*new_blk_size,
                    jblk*new_blk_size:(jblk+1)*new_blk_size] = find_replacement(rulebook, curr_block)
    return newgrid    

In [10]:
grid = pattern_to_matrix(start_pattern)
for i in range(2):
    grid = enhance(grid, test_rulebook)
assert(12 == grid.sum())

In [11]:
puzzle_rules = open('day21_input').read().split('\n')[:-1]
puzzle_rulebook = process_rules(puzzle_rules)
puzzle_iterations = 5

In [12]:
grid = pattern_to_matrix(start_pattern)
for i in range(puzzle_iterations):
    grid = enhance(grid, puzzle_rulebook)
grid.sum()

150.0

### puzzle 2 ###

In [13]:
puzzle_iterations2 = 18

Too many iterations for my current version. Let's see what happens if I have a hashable rulebook.

In [14]:
def process_rules2(rules):
    rulebook = {}
    for rule in rules:
        lhs, arrow, rhs = rule.partition('=>')
        lhs_arr = pattern_to_matrix(lhs.strip())
        rhs_arr = pattern_to_matrix(rhs.strip())
        for n in range(4):
            rotb = np.rot90(lhs_arr, n)
            rot_flip = np.fliplr(rotb)
            rulebook[tuple(rotb.flatten())] = rhs_arr
            rulebook[tuple(rot_flip.flatten())] = rhs_arr
    return rulebook

In [15]:
puzzle_rulebook_dict = process_rules2(puzzle_rules)

In [16]:
def enhance2(grid, rulebook):
    size = len(grid)
    if size % 2 == 0:
        blk_size = 2
        new_blk_size = 3
    elif size % 3 == 0:
        blk_size = 3
        new_blk_size = 4
    else:
        raise ValueError('Nonsensical grid size: {}'.format(size))
    num_blocks = len(grid)//blk_size
    # new grid is num_blocks × num_blocks, where each block is new_blk_size×new_blk_size
    newgrid = np.zeros((new_blk_size*num_blocks,new_blk_size*num_blocks))
    for iblk in range(num_blocks):
        for jblk in range(num_blocks):
            curr_block = grid[iblk*blk_size:(iblk+1)*blk_size,jblk*blk_size:(jblk+1)*blk_size]
            curr_hash = tuple(curr_block.flatten())
            newgrid[iblk*new_blk_size:(iblk+1)*new_blk_size,
                    jblk*new_blk_size:(jblk+1)*new_blk_size] = rulebook[curr_hash]
    return newgrid    

In [17]:
grid = pattern_to_matrix(start_pattern)
for i in range(puzzle_iterations2):
    grid = enhance2(grid, puzzle_rulebook_dict)
    print(i, len(grid))
grid.sum()

0 4
1 6
2 9
3 12
4 18
5 27
6 36
7 54
8 81
9 108
10 162
11 243
12 324
13 486
14 729
15 972
16 1458
17 2187


2606275.0